In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Rajendra c\\OneDrive\\Desktop\\Projects\\Text_sumarrizer_project\\research'

In [3]:
os.chdir("C:\\Users\\Rajendra c\\OneDrive\\Desktop\\Projects\\Text_sumarrizer_project")



In [4]:
%pwd

'C:\\Users\\Rajendra c\\OneDrive\\Desktop\\Projects\\Text_sumarrizer_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config.root_dir),  # Convert to Path
            data_path=Path(config.data_path),  # Convert to Path
            model_ckpt=str(config.model_ckpt),  # Keep as string for model loading
            num_train_epochs=int(params.num_train_epochs),  # Convert to int
            warmup_steps=int(params.warmup_steps),  # Convert to int
            per_device_train_batch_size=int(params.per_device_train_batch_size),  # Convert to int
            weight_decay=float(params.weight_decay),  # Convert to float
            logging_steps=int(params.logging_steps),  # Convert to int
            evaluation_strategy=str(params.evaluation_strategy),  # Keep as string
            eval_steps=int(params.eval_steps),  # Convert to int
            save_steps=float(params.save_steps),  # Convert to float
            gradient_accumulation_steps=int(params.gradient_accumulation_steps)  # Convert to int
        )

        return model_trainer_config

In [8]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch
import time
from datetime import datetime, timedelta

c:\Users\Rajendra c\OneDrive\Desktop\Projects\Text_sumarrizer_project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        start_time = time.time()
        
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {device}")
        
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, 
            num_train_epochs=self.config.num_train_epochs,
            warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size,
            per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay,
            logging_steps=self.config.logging_steps,
            eval_strategy=self.config.evaluation_strategy,  # Changed from evaluation_strategy to eval_strategy
            eval_steps=self.config.eval_steps,
            save_steps=self.config.save_steps,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        # Calculate estimated time
        num_epochs = self.config.num_train_epochs
        num_training_steps = len(dataset_samsum_pt["train"]) // self.config.per_device_train_batch_size * num_epochs
        print(f"\nTraining Config:")
        print(f"Number of epochs: {num_epochs}")
        print(f"Batch size: {self.config.per_device_train_batch_size}")
        print(f"Training samples: {len(dataset_samsum_pt['train'])}")
        print(f"Total training steps: {num_training_steps}")
        
        trainer = Trainer(
            model=model_pegasus, 
            args=trainer_args,
            tokenizer=tokenizer, 
            data_collator=seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["train"], 
            eval_dataset=dataset_samsum_pt["validation"]
        )
        
        print("\nStarting training...")
        trainer.train()
        
        end_time = time.time()
        training_time = end_time - start_time
        training_time_str = str(timedelta(seconds=int(training_time)))
        
        print(f"\nTraining completed!")
        print(f"Total training time: {training_time_str}")
        
        # Save model and tokenizer
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))
        
        return {
            "training_time": training_time_str,
            "device_used": device,
            "num_epochs": num_epochs,
            "total_steps": num_training_steps
        }

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    
    # Get training stats
    training_stats = model_trainer.train()
    
    print("\nTraining Summary:")
    print(f"Device used: {training_stats['device_used']}")
    print(f"Total epochs: {training_stats['num_epochs']}")
    print(f"Total steps: {training_stats['total_steps']}")
    print(f"Total time taken: {training_stats['training_time']}")
    
except Exception as e:
    raise e

[2025-09-05 15:46:05,291: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-05 15:46:05,291: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-05 15:46:05,298: INFO: common: created directory at: artifacts]
[2025-09-05 15:46:05,298: INFO: common: created directory at: artifacts/model_trainer]
Using device: cpu


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Training Config:
Number of epochs: 1
Batch size: 1
Training samples: 14732
Total training steps: 14732


C:\Users\Rajendra c\AppData\Local\Temp\ipykernel_6108\2425055447.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



Starting training...


c:\Users\Rajendra c\OneDrive\Desktop\Projects\Text_sumarrizer_project\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss


KeyboardInterrupt: 